In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)
cap = cv2.VideoCapture('/dev/video2')
ret, frame = cap.read()

dx = cv2.Sobel(frame, cv2.CV_32F, 1, 0)
dy = cv2.Sobel(frame, cv2.CV_32F, 0, 1)

mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0, 255).astype(np.uint8)

dst = np.zeros(frame.shape[:2], np.uint8)
_, dst = cv2.threshold(mag, 120, 255, cv2.THRESH_BINARY)

#cv2.imshow('src', src)
#cv2.imshow('mag', mag)
cv2.imshow('dst', dst)
cv2.waitKey()

cap.release()
cv2.destroyAllWindows()

In [5]:
#src = cv2.imread('./data/bricks.jpg', cv2.IMREAD_GRAYSCALE)
cap = cv2.VideoCapture('/dev/video2')
while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.Canny(frame, 50, 175)

    #cv2.imshow('src', src)
    cv2.imshow('canny', frame)
    key = cv2.waitKey(1)

    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
#src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)
cap = cv2.VideoCapture('/dev/video2')
while True:
    ret, frame = cap.read()

    if not ret:
        break

    edges = cv2.Canny(frame, 50, 150)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180., 160,
                            minLineLength=50, maxLineGap=5)

    frame = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    if lines is not None:
        for i in range(lines.shape[0]):
            pt1 = (lines[i][0][0], lines[i][0][1])
            pt2 = (lines[i][0][2], lines[i][0][3])
            cv2.line(frame, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

    #cv2.imshow('src', src)
    cv2.imshow('dst', frame)

    key = cv2.waitKey(1)

    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [7]:
src = cv2.imread('./data/dial.jpg')

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)

def on_trackbar(pos):
    rmin = cv2.getTrackbarPos('minRadius', 'img')
    rmax = cv2.getTrackbarPos('maxRadius', 'img')
    th = cv2.getTrackbarPos('threshold', 'img')

    circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50,
                               param1=120, param2=th, minRadius=rmin,
                               maxRadius=rmax)
    
    dst = src.copy()
    if circles is not None:
        for i in range(circles.shape[1]):
            cx, cy, radius = circles[0][i]
            cv2.circle(dst, (int(cx), int(cy)), int(radius), 
                       (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('img', dst)

cv2.imshow('img', src)
cv2.createTrackbar('minRadius', 'img', 0, 100, on_trackbar)
cv2.createTrackbar('maxRadius', 'img', 0, 150, on_trackbar)
cv2.createTrackbar('threshold', 'img', 0, 100, on_trackbar)
cv2.setTrackbarPos('minRadius', 'img', 10)
cv2.setTrackbarPos('maxRadius', 'img', 80)
cv2.setTrackbarPos('threshold', 'img', 40)
cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/hough.cpp:2296: error: (-211:One of the arguments' values is out of range) acc_threshold must be a positive number in function 'HoughCircles'


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/hough.cpp:2296: error: (-211:One of the arguments' values is out of range) acc_threshold must be a positive number in function 'HoughCircles'


In [7]:
src = cv2.imread('./data/coins1.jpg')

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)

circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50,
                               param1=150, param2=40, minRadius=20,
                               maxRadius=80)

sum_of_money = 0
dst = src.copy()
if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = circles[0][i]
        cv2.circle(dst, (int(cx), int(cy)), int(radius), 
                       (0, 0, 255), 2, cv2.LINE_AA)
        
        x1 = int(cx - radius)
        y1 = int(cy - radius)
        x2 = int(cx + radius)
        y2 = int(cy + radius)
        radius = int(radius)

        crop = dst[y1:y2, x1:x2, :]
        ch, cw = crop.shape[:2]

        mask = np.zeros((ch, cw), np.uint8)
        cv2.circle(mask, (cw//2, ch//2), radius, 255, -1)

        hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        hue, _, _ = cv2.split(hsv)
        hue_shift = (hue + 40) % 180
        mean_of_hue = cv2.mean(hue_shift, mask)[0]

        #plt.plot(hue_shift)

        won = 100
        if mean_of_hue < 53:
            won = 10

        sum_of_money += won

        cv2.putText(crop, str(won), (20, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, (255, 0, 0), 2, cv2.LINE_AA)

cv2.putText(dst, str(sum_of_money) + 'won', (40, 80), cv2.FONT_HERSHEY_SIMPLEX,
                    2, (255, 0, 0), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()